In [20]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
from __future__ import unicode_literals

import pandas
import numpy
from instrumentz import contract as icontract
from yamm import stats

In [2]:
def convert_to_mins(p):
    tup = p.to_timestamp().timetuple()
    return (tup.tm_hour * 60) + tup.tm_min

In [11]:
contract_name = 'CLF16_COMDTY'
start = pandas.Period('2015-12-14', freq='B')
end = pandas.Period('2015-12-18', freq='B')
mins_spec = (15, 120)

In [12]:
index = pandas.period_range(start=(start.asfreq('D')-1), end=end, freq='T')[1:]
index

PeriodIndex(['2015-12-14 00:00', '2015-12-14 00:01', '2015-12-14 00:02',
             '2015-12-14 00:03', '2015-12-14 00:04', '2015-12-14 00:05',
             '2015-12-14 00:06', '2015-12-14 00:07', '2015-12-14 00:08',
             '2015-12-14 00:09', 
             ...
             '2015-12-18 23:50', '2015-12-18 23:51', '2015-12-18 23:52',
             '2015-12-18 23:53', '2015-12-18 23:54', '2015-12-18 23:55',
             '2015-12-18 23:56', '2015-12-18 23:57', '2015-12-18 23:58',
             '2015-12-18 23:59'],
            dtype='int64', length=7200, freq='T')

In [14]:
df = icontract.factory(contract_name).get_vwap_data_df(index).loc[:, ('value', 'volume')]
df = df[~numpy.isnan(df.value)]
df.loc[:, 'mins'] = [convert_to_mins(p) for p in df.index]
df.head()

,value,volume,mins
2015-12-14 00:00,35.538957,163,0
2015-12-14 00:01,35.577458,59,1
2015-12-14 00:02,35.597000,10,2
2015-12-14 00:03,35.577073,41,3
2015-12-14 00:04,35.579032,31,4


In [16]:
foo = df.copy()
mins = mins_spec[0]
foo.loc[:, 'date'] = [dt.date() for dt in foo.index.to_timestamp()]
grps = (foo.mins // mins).values
foo.loc[:, 'grp'] = grps
foo.head()

,value,volume,mins,date,grp
2015-12-14 00:00,35.538957,163,0,2015-12-14,0
2015-12-14 00:01,35.577458,59,1,2015-12-14,0
2015-12-14 00:02,35.597000,10,2,2015-12-14,0
2015-12-14 00:03,35.577073,41,3,2015-12-14,0
2015-12-14 00:04,35.579032,31,4,2015-12-14,0


In [26]:
grouping = foo.loc[:, ('date', 'value', 'volume', 'grp')].groupby('grp')
for grp, fam in grouping:
    florb = []
    for dt, faz in fam.groupby('date'):
        florb.append([dt, stats.wmean(faz['value'], faz['volume']), int(faz['volume'].sum())])
    florb = pandas.DataFrame(florb, columns=['date', 'value', 'volume']).set_index('date')
florb

,value,volume
date,,
2015-12-14,36.184576,177
2015-12-15,36.808771,179
2015-12-16,35.665355,155
2015-12-17,34.821591,44
